In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [51]:
cd "/content/drive/MyDrive/mlh-fellowship-hack"

/content/drive/MyDrive/mlh-fellowship-hack


In [52]:
ls

booksummaries/        model.py                __pycache__/
booksummaries.tar.gz  multilabel_pipeline.py


In [63]:
!pip install ibm_watson

In [ ]:
!tar xvzf booksummaries.tar.gz

booksummaries/
booksummaries/README
booksummaries/booksummaries.txt


In [64]:
import csv
data = []

with open("booksummaries/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in reader:
        data.append(row)

In [65]:
len(data)

16559

In [67]:
import pandas as pd

book_wiki = []
book_title = []
book_author = []
publication_date = []
book_genre = []
book_summary = []
book_img = []
for i in data:
    book_wiki.append("https://en.wikipedia.org/?curid="+str(i[0]))
    book_title.append(i[2])
    book_author.append(i[3])
    publication_date.append(i[4])
    book_genre.append(i[5])
    book_summary.append(i[6])
    book_img.append("https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh3_t5Djz8od-jE=/0x86:1192x710/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/22312759/rickroll_4k.jpg")

df = pd.DataFrame({'Wikipedia Link': book_wiki, 'Book Title': book_title, 'Author': book_author, 'Date of Publication': publication_date, 'Genre': book_genre, 'Raw Summary':book_summary, 'Image URL':book_img})
df.head()

,Wikipedia Link,Book Title,Author,Date of Publication,Genre,Raw Summary,Image URL
0,https://en.wikipedia.org/?curid=620,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...",https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...
1,https://en.wikipedia.org/?curid=843,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...",https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...
2,https://en.wikipedia.org/?curid=986,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...
3,https://en.wikipedia.org/?curid=1756,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...
4,https://en.wikipedia.org/?curid=2080,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...


In [68]:
import json
import re
df.isna().sum()

df = df.drop(df[df['Genre'] == ''].index)
df = df.drop(df[df['Raw Summary'] == ''].index)


genres_cleaned = []
for i in df['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))
df['Genres'] = genres_cleaned

def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text



df['Summary'] = df['Raw Summary'].apply(lambda x: clean_summary(x))
df.drop(['Raw Summary', 'Genre'], axis=1, inplace=True)
df.head()


,Wikipedia Link,Book Title,Author,Date of Publication,Image URL,Genres,Summary
0,https://en.wikipedia.org/?curid=620,Animal Farm,George Orwell,1945-08-17,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...
1,https://en.wikipedia.org/?curid=843,A Clockwork Orange,Anthony Burgess,1962,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...
2,https://en.wikipedia.org/?curid=986,The Plague,Albert Camus,1947,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...
4,https://en.wikipedia.org/?curid=2080,A Fire Upon the Deep,Vernor Vinge,,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...
5,https://en.wikipedia.org/?curid=2152,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[War novel, Roman à clef]",the book tells the story of paul b umer a germ...


In [70]:
apikey = ''
url = ''
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(apikey)
ta = ToneAnalyzerV3(version='2017-09-21', authenticator=authenticator)
ta.set_service_url(url)
fin = []

def get_emotions(text):
  try:
    res = ta.tone(text).get_result()
    res = res['document_tone']['tones']
    ans = []
    for item in res:
      ans.append({item['tone_id']:item['score']})
    ans = {'emotions':ans}
    fin.append(ans)
    return ans
  except Exception as e:
    ans = {'emotions':[]}
    fin.append(ans)
    return ans

df['Emotions'] = df['Summary'].apply(lambda x: get_emotions(x))

In [71]:
df.to_csv('database.csv')

In [72]:
df.to_json('database.json', orient = 'split', compression = 'infer')

In [74]:
df.tail(10)

,Wikipedia Link,Book Title,Author,Date of Publication,Image URL,Genres,Summary,Emotions
16541,https://en.wikipedia.org/?curid=35916725,Nevermore: The Final Maximum Ride Adventure,James Patterson,2012-08-06,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,[Young adult literature],the prologue is angels vision of maxs death th...,{'emotions': [{'sadness': 0.593374}]}
16542,https://en.wikipedia.org/?curid=35993963,The Bourne Imperative,Robert Ludlum,2012-06-05,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Thriller, Spy fiction]",for a more detailed background of the main cha...,"{'emotions': [{'sadness': 0.659248}, {'analyti..."
16544,https://en.wikipedia.org/?curid=36126507,Heaven,V. C. Andrews,1985-10-15,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Gothic fiction, Young adult literature, Fiction]",heaven leigh casteel is a fourteen year old gi...,{'emotions': [{'sadness': 0.87905}]}
16545,https://en.wikipedia.org/?curid=36126601,Dark Angel,V. C. Andrews,1986-11,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Gothic fiction, Young adult literature]",after the events of heaven the first book in t...,{'emotions': [{'sadness': 0.792429}]}
16546,https://en.wikipedia.org/?curid=36126792,Gates of Paradise,V. C. Andrews,1989-06,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Gothic fiction, Young adult literature]",a novel about annie stonewall the daughter of ...,{'emotions': [{'sadness': 0.526255}]}
16549,https://en.wikipedia.org/?curid=36372465,The Third Lynx,Timothy Zahn,2007,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,[Science Fiction],the story starts with former government agent ...,{'emotions': [{'fear': 0.762139}]}
16551,https://en.wikipedia.org/?curid=36534061,Remote Control,Andy McNab,1997,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Thriller, Fiction, Suspense]",the series follows the character of nick stone...,"{'emotions': [{'sadness': 0.718305}, {'analyti..."
16555,https://en.wikipedia.org/?curid=37054020,Transfer of Power,Vince Flynn,2000-06-01,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Thriller, Fiction]",the reader first meets rapp while he is doing ...,{'emotions': []}
16556,https://en.wikipedia.org/?curid=37122323,Decoded,Jay-Z,2010-11-16,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,[Autobiography],the book follows very rough chronological orde...,"{'emotions': [{'sadness': 0.512098}, {'analyti..."
16558,https://en.wikipedia.org/?curid=37159503,Poor Folk,Fyodor Dostoyevsky,1846,https://cdn.vox-cdn.com/thumbor/HWPOwK-35K4Zkh...,"[Epistolary novel, Speculative fiction]",makar devushkin and varvara dobroselova are se...,"{'emotions': [{'sadness': 0.927617}, {'analyti..."
